In [1]:
from IPython.display import Image

# Lab 2 - Logistic Regression with MNIST


In this tutorial we will build and train a Multiclass Logistic Regression model using the MNIST data. 

The MNIST data comprises of hand-written digits with little background noise making it a standard dataset to create, experiment and learn deep learning models with reasonably small comptuing resources.



In [2]:
# Figure 1
Image(url= "http://3.bp.blogspot.com/_UpN7DfJA0j4/TJtUBWPk0SI/AAAAAAAAABY/oWPMtmqJn3k/s1600/mnist_originals.png", width=200, height=200)


[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) (LR) is a fundamental machine learning technique that uses a linear weighted combination of features and generates probability-based predictions of different classes.  
​
There are two basic forms of LR: **Binary LR** (with a single output that can predict two classes) and **multinomial LR** (with multiple outputs, each of which is used to predict a single class).  

![LR-forms](http://www.cntk.ai/jup/cntk103b_TwoFormsOfLR-v3.png)

In **Binary Logistic Regression** (see top of figure above), the input features are each scaled by an associated weight and summed together.  The sum is passed through a squashing (aka activation) function and generates an output in [0,1].  This output value (which can be thought of as a probability) is then compared with a threshold (such as 0.5) to produce a binary label (0 or 1).  This technique supports only classification problems with two output classes, hence the name binary LR.  In the binary LR example shown above, the [sigmoid][] function is used as the squashing function.

[sigmoid]: https://en.wikipedia.org/wiki/Sigmoid_function

In **Multiclass Linear Regression** (see bottom of figure above), 2 or more output nodes are used, one for each output class to be predicted.  Each summation node uses its own set of weights to scale the input features and sum them together. Instead of passing the summed output of the weighted input features through a sigmoid squashing function, the output is often passed through a `softmax` function (which in addition to squashing, like the `sigmoid`, the `softmax` normalizes each nodes' output value using the sum of all unnormalized nodes). 

$$ h(\textbf{z})_i = \frac{e^{z_i}}{\sum_{k=1}^C{e^{z_k}}} \text{,  where  } \textbf{z} = \textbf{W} \times \textbf{x}  + \textbf{b} $$

In this tutorials, we will use multinomial LR for classifying the MNIST digits (0-9) using 10 output nodes (1 for each of our output classes).



The figure below summarizes the model in the context of the MNIST data.

![mnist-LR](https://www.cntk.ai/jup/cntk103b_MNIST_LR.png)

The trainer strives to reduce the `loss` function by different optimization approaches, [Stochastic Gradient Descent][] (`sgd`) being one of the most popular one. Typically, one would start with random initialization of the model parameters. The `sgd` optimizer would calculate the `loss` or error between the predicted label against the corresponding ground-truth label and using [gradient-decent][] generate a new set model parameters in a single iteration. 

The aforementioned model parameter update using a single observation at a time is attractive since it does not require the entire data set (all observation) to be loaded in memory and also requires gradient computation over fewer datapoints, thus allowing for training on large data sets. However, the updates generated using a single observation sample at a time can vary wildly between iterations. An intermediate ground is to load a small set of observations and use an average of the `loss` or error from that set to update the model parameters. This subset is called a *minibatch*.

With minibatches, we often sample observation from the larger training dataset. We repeat the process of model parameters update using different combination of training samples and over a period of time minimize the `loss` (and the error). When the incremental error rates are no longer changing significantly or after a preset number of maximum minibatches to train, we claim that our model is trained.

One of the key optimization parameter is called the `learning_rate`. For now, we can think of it as a scaling factor that modulates how much we change the parameters in any iteration. 

[optimization]: https://en.wikipedia.org/wiki/Category:Convex_optimization
[Stochastic Gradient Descent]: https://en.wikipedia.org/wiki/Stochastic_gradient_descent
[gradient-decent]: http://www.statisticsviews.com/details/feature/5722691/Getting-to-the-Bottom-of-Regression-with-Gradient-Descent.html

## Initialize environment

In [13]:
# Import the relevant components

import sys
import os
import time
import numpy as np
import cntk as C
from cntk.logging.progress_print import ProgressPrinter


# Select the right target device when this notebook is being tested:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))


## Data reading

In this tutorial we are using the MNIST data pre-processed to follow CNTK CTF format. The dataset has 50,000 training images, 10,000 validation images, and 10,000 test images with each image being 28 x 28 pixels. Thus the number of features is equal to 784 (= 28 x 28 pixels), 1 per pixel. The variable `num_output_classes` is set to 10 corresponding to the number of digits (0-9) in the dataset.

The data is in the following format:

    |labels 0 0 0 0 0 0 0 1 0 0 |features 0 0 0 0 ... 
                                                  (784 integers each representing a pixel)
    


In [4]:
# Ensure we always get the same amount of randomness
np.random.seed(0)

# Define the data dimensions
input_dim = 784
num_output_classes = 10

# Read a CTF formatted text (as mentioned above) using the CTF deserializer from a file
def create_reader(path, is_training, input_dim, num_label_classes):
    labelStream = C.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False)
    featureStream = C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserializer = C.io.CTFDeserializer(path, C.io.StreamDefs(labels = labelStream, features = featureStream))
    return C.io.MinibatchSource(deserializer,
       randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)


## Model training



### Set up a computational network


In [5]:
# Define a computational network for multi-class logistic regression
def create_mlr_model(features, output_dim):
    input_dim = features.shape[0]
    weight_param = C.parameter(shape=(input_dim, output_dim))
    bias_param = C.parameter(shape=(output_dim))
    return C.times(features, weight_param) + bias_param

# Define features and labels
input = C.input(input_dim)
label = C.input(num_output_classes)

# Scale the input to 0-1 range by dividing each pixel by 255.
z = create_mlr_model(input/255.0, num_output_classes)


### Define a loss and error functions



In [6]:
# Define loss and error functions
loss = C.cross_entropy_with_softmax(z, label)
label_error = C.classification_error(z, label)


### Configure a trainer with the SGD learner


In [7]:
# Instantiate the trainer object to drive the model training
learning_rate = 0.2
lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
learner = C.sgd(z.parameters, lr_schedule)
progress_printer = ProgressPrinter(500)
trainer = C.Trainer(z, (loss, label_error), [learner], [progress_printer])


### Run the trainer

In [11]:
# Create the reader to training data set
train_file = "../../Data/MNIST_train.txt"
reader_train = create_reader(train_file, True, input_dim, num_output_classes)


In [14]:
# Initialize the parameters for the trainer
minibatch_size = 64
num_samples_per_sweep = 60000
num_sweeps_to_train_with = 10
num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size

# Map the data streams to the input and labels.
input_map = {
    label  : reader_train.streams.labels,
    input  : reader_train.streams.features
}

start_time = time.time()
# Run the trainer on and perform model training
for i in range(0, int(num_minibatches_to_train)):
    # Read a mini batch from the training data file
    data = reader_train.next_minibatch(minibatch_size, input_map = input_map)
    trainer.train_minibatch(data)

print(time.time() - start_time)


Learning rate per minibatch: 0.2
 Minibatch[   1- 500]: loss = 0.469908 * 32000, metric = 12.47% * 32000;
 Minibatch[ 501-1000]: loss = 0.343888 * 32000, metric = 9.69% * 32000;
 Minibatch[1001-1500]: loss = 0.315138 * 32000, metric = 8.85% * 32000;
 Minibatch[1501-2000]: loss = 0.299228 * 32000, metric = 8.44% * 32000;
 Minibatch[2001-2500]: loss = 0.295266 * 32000, metric = 8.39% * 32000;
 Minibatch[2501-3000]: loss = 0.296087 * 32000, metric = 8.19% * 32000;
 Minibatch[3001-3500]: loss = 0.285083 * 32000, metric = 8.03% * 32000;
 Minibatch[3501-4000]: loss = 0.279358 * 32000, metric = 7.78% * 32000;
 Minibatch[4001-4500]: loss = 0.279786 * 32000, metric = 7.76% * 32000;
 Minibatch[4501-5000]: loss = 0.279303 * 32000, metric = 7.76% * 32000;
 Minibatch[5001-5500]: loss = 0.273061 * 32000, metric = 7.63% * 32000;
 Minibatch[5501-6000]: loss = 0.273658 * 32000, metric = 7.65% * 32000;
 Minibatch[6001-6500]: loss = 0.266554 * 32000, metric = 7.44% * 32000;
 Minibatch[6501-7000]: loss = 

## Model Evaluation



In [15]:
# Read the validation data
test_file = "../../Data/MNIST_validate.txt"
reader_test = create_reader(test_file, False, input_dim, num_output_classes)


In [16]:

test_input_map = {
    label  : reader_test.streams.labels,
    input  : reader_test.streams.features,
}

# Test data for trained model
test_minibatch_size = 512
num_samples = 10000
num_minibatches_to_test = num_samples // test_minibatch_size
test_result = 0.0

for i in range(num_minibatches_to_test):
    data = reader_test.next_minibatch(test_minibatch_size, input_map = test_input_map)
    eval_error = trainer.test_minibatch(data)
    test_result = test_result + eval_error

# Average of evaluation errors of all test minibatches
print("Average validation error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))


Average validation error: 8.15%


# Hackathon

Try to improve the performance of the model. 

Hints:
- Play with the learning rate, minibatch size and the number of sweeps
- You can look at regularization - check `l1_regularization` and `l2_regularization` hyper parameters of the `sgd` learner



## Final testing

### Run the following cell after you have fine tuned your model.

### Don't cheat. Don't use the final test data during model fine tuning

In [18]:
# Final testing
# Read the test data set
test_file = "../../Data/MNIST_test.txt"
reader_test = create_reader(test_file, False, input_dim, num_output_classes)

test_input_map = {
    label  : reader_test.streams.labels,
    input  : reader_test.streams.features,
}

test_minibatch_size = 512
num_samples = 10000
num_minibatches_to_test = num_samples // test_minibatch_size
test_result = 0.0

for i in range(num_minibatches_to_test):
    data = reader_test.next_minibatch(test_minibatch_size, input_map = test_input_map)
    eval_error = trainer.test_minibatch(data)
    test_result = test_result + eval_error

# Average of evaluation errors of all test minibatches
print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))


Average test error: 7.43%
